In [1]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd

from conllu_to_df import process_file
from extract_features import extract_features
import numpy as np

In [2]:
train_file = "../UP-1.0/input/en_ewt-up-train.conllu"
dev_file = "../UP-1.0/input/en_ewt-up-dev.conllu"
test_file = "../UP-1.0/input/en_ewt-up-test.conllu"

out_train_file = '../UP-1.0/output/train.csv'
out_test_file = '../UP-1.0/output/test.csv'
out_dev_file = '../UP-1.0/output/dev.csv'

feat_train_file = out_train_file.replace('output', 'features')
feat_test_file = out_test_file.replace('output', 'features')
feat_dev_file = out_dev_file.replace('output', 'features')

In [3]:
# process_file(dev_file, out_dev_file)
# process_file(test_file, out_test_file)
# process_file(train_file, out_train_file)

In [4]:
# extract_features(out_dev_file, feat_dev_file)
# extract_features(out_train_file, feat_train_file)
# extract_features(out_test_file, feat_test_file)

In [5]:
vectorizer = DictVectorizer()

In [6]:
def extract_features_and_labels(trainingfile):
    features = []
    gold = []
    with open(trainingfile, 'r', encoding='utf-8') as infile:
        for line in infile.readlines()[1:]:
            components = line.rstrip('\n').split('\t')
            if len(components) > 0:
                token = components[0]
                pos = components[1]
                lemma = components[2]
                dependency = components[3]
                head = components[4]
                head_pos = components[5]
                morph_feat = components[6]
                ent_type = components[7]
                ent_dep = components[8]
                voice = components[9]
                gold_a = components[-1]
                feature_dict = {'Token': token,
                                'PoS': pos,
                                'Lemma': lemma, 
                                'dependency': dependency, 
                                'head': head, 
                                'head pos': head_pos,
                                'morph feature': morph_feat,
                                'entity type': ent_type,
                                'entity dep': ent_dep,
                                'voice': voice}
                features.append(feature_dict)
                gold.append(gold_a)
    return features, gold

In [8]:
vec = DictVectorizer()

# X_train, y_train = extract_features_and_labels(feat_train_file)
# X_train_vec = vec.fit_transform(X_train)

# X_dev, y_dev = extract_features_and_labels(feat_dev_file)
# X_dev_vec = vec.transform(X_dev)

X_test, y_test = extract_features_and_labels(feat_test_file)
X_test_vec = vec.transform(X_test)

IndexError: list index out of range

In [8]:
model = LogisticRegression(solver='liblinear', max_iter=500)

model.fit(X_train_vec, y_train)

c:\ProgramData\Anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1222: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  warnings.warn(


LogisticRegression(max_iter=500, n_jobs=4, solver='liblinear')

In [9]:
y_pred = model.predict(X_test_vec)

In [12]:
print("METRICS")
print('precision', precision_score(y_test, y_pred, average='weighted'), sep='\t')
print('recall', recall_score(y_test, y_pred, average='weighted'), sep='\t')
print('f1', f1_score(y_test, y_pred, average='weighted'), sep='\t')

METRICS


c:\ProgramData\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


precision	0.7255792691324848
recall	0.8517638044343153
f1	0.7836242089238771
